In [26]:
import numpy as np
from collections import deque

In [61]:
class Binary_node():
    def  __init__(self,parent,value):
        self.parent = parent
        self.value = value
        self.left_child = None
        self.right_child = None
        
    def __str__(self):
        left_child_value = self.left_child.value if self.left_child is not None else 'None'
        right_child_value = self.right_child.value if self.right_child is not None else 'None'
        parent_value = self.get_parent().value if not self.is_root() else None
        return f'parent:{parent_value},value:{self.value},index:{self.index}'
    
    def set_parent(self, parent):
        self.parent = parent
    
    def set_child(self, child, is_right:bool):
        child.set_parent(self)
        if is_right:
            self.right_child = child
        else:
            self.left_child = child
    
    def get_parent(self):
        return self.parent
    
    def get_child(self,is_right:bool):
        if is_right:
            return self.right_child
        else:
            return self.left_child
    
    def is_root(self):
        return (self.parent is None)
    
    def has_child(self,is_right):
        if is_right:
            return self.right_child is not None
        else:
            return self.left_child is not None
        

class Binary_tree():
    def  __init__(self,root_value):
        self.root = Binary_node(parent=None,value=root_value)
    
    def get_root(self):
        return self.root
    
    def set_root(self,new_root):
        self.root = new_root

In [62]:
class Cartesian_node(Binary_node):
    def __init__(self,parent,value,index):
        super().__init__(parent,value)
        self.index = index
    
    def get_index(self):
        return self.index
    
    def set_index(self,index):
        self.index = index
        
        
class Cartesian_tree(Binary_tree):
    def __init__(self,A:list):
        self.set_root(Cartesian_node(None,A[0],0))
        self.size = len(A)
        for i in range(1,len(A)):
            node_on_rightmost_path = self.get_root()
            while node_on_rightmost_path.has_child(is_right=True):
                if A[i] < node_on_rightmost_path.value and node_on_rightmost_path.is_root:
                    break
                next_node = node_on_rightmost_path.get_child(is_right=True)
                if node_on_rightmost_path.value <= A[i] < next_node.value:
                    break
                node_on_rightmost_path = next_node
                
                
            if not node_on_rightmost_path.has_child(is_right=True):
                new_node = Cartesian_node(node_on_rightmost_path,A[i],i)
                node_on_rightmost_path.set_child(new_node,is_right=True)
            elif node_on_rightmost_path.is_root() and A[i] < node_on_rightmost_path.value:
                new_node = Cartesian_node(None,A[i],i)
                new_node.set_child(node_on_rightmost_path,is_right=False)
                self.set_root(new_node)
            else:
                new_node = Cartesian_node(node_on_rightmost_path,A[i],i)
                if node_on_rightmost_path.has_child(is_right=True):
                    new_node.set_child(node_on_rightmost_path.get_child(is_right=True), is_right=False)
                node_on_rightmost_path.set_child(new_node,is_right=True)
        

In [63]:
def show_tree(c: Cartesian_tree):
    nodes_to_search = deque([[c.root,1]])
    current_depth=0
    while len(nodes_to_search) > 0:
        node,depth = nodes_to_search.popleft()
        if depth != current_depth:
            temp = "\n" if current_depth!=0 else ""
            print(f'{temp}depth:{depth}')
            current_depth = depth
        print(node)
        if node.has_child(is_right=False):
            nodes_to_search.append([node.get_child(is_right=False),depth+1])
        if node.has_child(is_right=True):
            nodes_to_search.append([node.get_child(is_right=True),depth+1])

In [75]:
c = Cartesian_tree([2,4,3,5,1,8,6,7,9])

In [76]:
show_tree(c)

depth:1
parent:None,value:1,index:4

depth:2
parent:1,value:2,index:0
parent:1,value:6,index:6

depth:3
parent:2,value:3,index:2
parent:6,value:8,index:5
parent:6,value:7,index:7

depth:4
parent:3,value:4,index:1
parent:3,value:5,index:3
parent:7,value:9,index:8


In [70]:
x = np.random.randint(1,10,20)
print(x)
c = Cartesian_tree(x)
show_tree(c)

[8 6 3 4 5 4 4 4 3 8 6 2 8 4 1 7 2 7 8 1]
depth:1
parent:None,value:1,index:14

depth:2
parent:1,value:2,index:11
parent:1,value:1,index:19

depth:3
parent:2,value:3,index:2
parent:2,value:4,index:13
parent:1,value:2,index:16

depth:4
parent:3,value:8,index:0
parent:3,value:3,index:8
parent:4,value:8,index:12
parent:2,value:7,index:15
parent:2,value:7,index:17

depth:5
parent:8,value:6,index:1
parent:3,value:4,index:3
parent:3,value:6,index:10
parent:7,value:8,index:18

depth:6
parent:4,value:4,index:5
parent:6,value:8,index:9

depth:7
parent:4,value:5,index:4
parent:4,value:4,index:6

depth:8
parent:4,value:4,index:7


In [71]:
def make_excess_array_from_Cartesian_tree(tree:Cartesian_tree):
    n = tree.size
    array = np.zeros(2*n+1,dtype='int')
    node_to_search = tree.get_root()
    depth, search_index = 1,1
    left_already_searched = False
    right_already_searched = False
    while True:
        if (not left_already_searched) and (not right_already_searched):
            array[search_index] = depth
        else:
            search_index -= 1
        if node_to_search.has_child(is_right=False) and not left_already_searched:
            node_to_search = node_to_search.get_child(is_right=False)
            left_already_searched = False
            right_already_searched = False
            search_index += 1
            depth += 1
            continue
        if node_to_search.has_child(is_right=True) and not right_already_searched:
            node_to_search = node_to_search.get_child(is_right=True)
            left_already_searched = False
            right_already_searched = False
            search_index += 1
            depth += 1
            continue
        if not node_to_search.has_child(is_right=True) or right_already_searched:
            search_index += 1
            array[search_index] = depth-1 # finish
            if node_to_search.is_root():
                break
            parent = node_to_search.get_parent()
            if parent.has_child(is_right=False) and parent.get_child(is_right=False)==node_to_search:
                right_already_searched = False
            if parent.has_child(is_right=True) and parent.get_child(is_right=True)==node_to_search:
                right_already_searched = True
            node_to_search = parent
            search_index += 1
            depth -= 1
            left_already_searched = True
    return array

In [77]:
make_excess_array_from_Cartesian_tree(c)

array([0, 1, 2, 3, 4, 3, 4, 3, 2, 1, 2, 3, 2, 3, 4, 3, 2, 1, 0])

In [78]:
def make_plus_minus_one_array(tree:Cartesian_tree):
    DFS_array = make_excess_array_from_Cartesian_tree(tree)
    B = ((DFS_array[1:]-DFS_array[:-1])==1).astype('int')
    return B

In [80]:
make_plus_minus_one_array(c)

array([1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0])